In [1]:
#import the packages 
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel      
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix



In [3]:
path = 'C:/Users/Pratik/Desktop/my codes/Predict the Correct Segment/file.csv'
seg = pd.read_csv(path,encoding='latin1')
categorical = seg.select_dtypes(exclude=['number']).columns
numerical = seg.select_dtypes(include=['number']).columns
#categorical columns
#categorical = [var for var in seg.columns if seg[var].dtype=='O']

#numerical columns
#numerical = [var for var in seg.columns if seg[var].dtype!='O']

df = pd.concat([seg[numerical], pd.get_dummies(seg[categorical])],axis=1) # get_dummies for one hot encoding for categorical variable
df.head()

,customer id,Nearest_distance_cosco,Net Sales,Age,children count,Martial status,Loyality_flag,coupon amount,Income,Orders groceries,...,Brands_ Shiseido Co,Brands_ Spectrum Brands Holdings,Brands_ Stanley Black & Decker,Brands_ Svenska Cellulosa AB SCA,Brands_ Swatch Group SA*,Brands_ Tyson Foods,Brands_ Unilever N,Brands_ VF Corp,Brands_ WH Group Ltd,Brands_ Whirlpool Corp
0,5102,4,367941,58,4,0,0,6,4628,6,...,0,0,0,0,0,0,0,0,0,0
1,3549,33,171305,98,1,0,0,43,4074,7,...,0,0,0,0,0,0,0,0,0,0
2,5885,28,173439,18,2,0,0,42,4910,5,...,0,0,0,0,0,0,0,0,0,0
3,7381,40,997988,87,3,0,1,38,3297,5,...,0,0,0,0,0,0,0,0,0,0
4,2713,5,898554,31,2,0,0,24,4940,4,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# independent variable
X=df.drop(['customer id','segments'], axis=1)

#dependent variable
y=df['segments']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, shuffle = True)

# Instantiate the XGBoost calssifier
classifier = xgb.XGBClassifier(random_state=2)
classifier.fit(X_train, y_train)

#Making prediction on test data
y_pred = classifier.predict(X_test)


f1 = f1_score(y_test, y_pred, average='macro')
print('f1_score:\n',f1)

#Finding the classification report
xgb_cr=classification_report(y_test,y_pred)
print('Classification report: \n', xgb_cr)



f1_score:
 0.1784048478126609
Classification report: 
               precision    recall  f1-score   support

           1       0.19      0.24      0.21       247
           2       0.19      0.10      0.14       288
           3       0.19      0.30      0.23       241
           4       0.13      0.13      0.13       231
           5       0.15      0.11      0.13       251
           6       0.23      0.22      0.23       242

   micro avg       0.18      0.18      0.18      1500
   macro avg       0.18      0.19      0.18      1500
weighted avg       0.18      0.18      0.18      1500



## gridsearchCV to increase the performance of our model.

In [7]:
from sklearn.model_selection import GridSearchCV
parameters={'learning_rate':[0.1,0.15,0.2,0.25,0.3],
            'max_depth':range(1,3)}

# instantiate the GridsearchCV 
grid_search = GridSearchCV(estimator = classifier,param_grid = parameters,n_jobs = -1,verbose=4)

# fit the model
grid_search = grid_search.fit(X_train, y_train)

# predict the result
grid_predictions =grid_search.predict(X_test)

#calculate the f1 score
grid_f1= f1_score(y_test, grid_predictions, average='macro')
print(grid_f1)

# calssification report
report=classification_report(y_test, grid_predictions)
print(report)


C:\Users\Pratik\Anaconda3\envs\greyatom\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.6min finished


0.15620295434336381
              precision    recall  f1-score   support

           1       0.21      0.23      0.22       247
           2       0.23      0.07      0.10       288
           3       0.17      0.44      0.25       241
           4       0.13      0.15      0.14       231
           5       0.16      0.09      0.11       251
           6       0.16      0.09      0.12       242

   micro avg       0.17      0.17      0.17      1500
   macro avg       0.18      0.18      0.16      1500
weighted avg       0.18      0.17      0.15      1500



As we can see that the f1_score descrease after applying the gridSeachCV. We have given you basic parameters in order to run the nodel quickly. You can do the hyperparameter tunning so that you will get the good result.

In [8]:
from sklearn.ensemble import RandomForestClassifier


# code starts here
model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

model_f1 = f1_score(y_test, y_pred, average='macro')
print('f1_score:\n',model_f1)

#Finding the classification report
model_report=classification_report(y_test,y_pred)
print('Classification report: \n', model_report)


# code ends here

C:\Users\Pratik\Anaconda3\envs\greyatom\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


f1_score:
 0.16822194242135355
Classification report: 
               precision    recall  f1-score   support

           1       0.18      0.28      0.22       247
           2       0.20      0.19      0.20       288
           3       0.16      0.18      0.17       241
           4       0.15      0.14      0.15       231
           5       0.14      0.10      0.12       251
           6       0.19      0.14      0.16       242

   micro avg       0.17      0.17      0.17      1500
   macro avg       0.17      0.17      0.17      1500
weighted avg       0.17      0.17      0.17      1500

